In [3]:
import os
import sys

sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"
os.environ["NUMBA_DEBUG_CACHE"] = "1"

In [4]:
import numpy as np
import pandas as pd
from my_stuff import BinanceFuturesTestKeys
from requests import get
from quantfreedom.enums import *
from quantfreedom.exchanges.binance_exchange.binance_futures import BinanceFutures

# mufex_main = Mufex(
#     api_key=MufexKeys.api_key,
#     secret_key=MufexKeys.secret_key,
#     use_test_net=False,
# )

binance_futures_test = BinanceFutures(
    api_key=BinanceFuturesTestKeys.api_key,
    secret_key=BinanceFuturesTestKeys.secret_key,
    use_test_net=True,
)

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
symbol = "BTCUSDT"
binance_futures_timeframe = "4m"

params = {
    "symbol": symbol,
    "interval": binance_futures_timeframe,
    "limit": 2,
}
url_start = "https://fapi.binance.com"
end_point = "/fapi/v1/klines"
entries = params.items()
params_as_path = "&".join("{key}={value}".format(
    key=x[0], value=x[1]) for x in entries if x[1] is not None)

get(url=url_start + end_point + "?" + params_as_path).json()

{'code': 0,
 'msg': "Service unavailable from a restricted location according to 'b. Eligibility' in https://www.binance.com/en/terms. Please contact customer service if you believe you received this message in error."}

In [37]:
symbol = "BTCUSDT"
binance_futures_timeframe = "4m"

binance_futures_test.get_candles_main_no_keys(symbol=symbol, timeframe=binance_futures_timeframe, limit=5)

ValueError: '4m' is not in list

In [27]:
candles = []
candles.appresponse)[:, :5]

[[1698358500000,
  '34179.10',
  '34198',
  '34169.30',
  '34177.60',
  '282.248',
  1698358799999,
  '9649287.42570',
  7197,
  '151.355',
  '5174417.22950',
  '0'],
 [1698358800000,
  '34177.60',
  '34184',
  '34169',
  '34171.20',
  '82.958',
  1698359099999,
  '2835198.31840',
  1377,
  '29.817',
  '1019015.68580',
  '0']]